Notebook for testing the general noisy system and plotting results. [Link to the chip specifications](https://www.bsc.es/supportkc/docs/Quantum/blue/)

## PSR

In [1]:
from qibochem.measurement.result import expectation_from_samples
import numpy as np


def evaluate(circuit, hamiltonian, nshots):
    if nshots is None:
        return hamiltonian.expectation(circuit().state())
    return expectation_from_samples(circuit, hamiltonian, nshots, group_pauli_terms="qwc")

In [54]:
def parameter_shift(
    circuit,
    hamiltonian,
    parameter_index,
    initial_state=None,
    scale_factor=1,
    nshots=None,
    nruns=1,
    repeated = False
):
    """In this method the parameter shift rule (PSR) is implemented.
    Given a circuit U and an observable H, the PSR allows to calculate the derivative
    of the expected value of H on the final state with respect to a variational
    parameter of the circuit.
    There is also the possibility of setting a scale factor. It is useful when a
    circuit's parameter is obtained by combination of a variational
    parameter and an external object, such as a training variable in a Quantum
    Machine Learning problem. For example, performing a re-uploading strategy
    to embed some data into a circuit, we apply to the quantum state rotations
    whose angles are in the form: theta' = theta * x, where theta is a variational
    parameter and x an input variable. The PSR allows to calculate the derivative
    with respect of theta' but, if we want to optimize a system with respect its
    variational parameters we need to "free" this procedure from the x depencency.
    If the `scale_factor` is not provided, it is set equal to one and doesn't
    affect the calculation.
    Args:
        circuit (:class:`qibo.models.circuit.Circuit`): custom quantum circuit.
        hamiltonian (:class:`qibo.hamiltonians.Hamiltonian`): target observable.
        parameter_index (int): the index which identifies the target parameter in the circuit.get_parameters() list
        initial_state ((2**nqubits) vector): initial state on which the circuit acts (default None).
        scale_factor (float): parameter scale factor (default None).
        repeated (bool): indicates if the gates that use the parameter have a translational symmetry
    Returns:
        np.float value of the derivative of the expectation value of the hamiltonian
        with respect to the target variational parameter.
    Example:
        .. testcode::
            import qibo
            import numpy as np
            from qibo import hamiltonians, gates
            from qibo.models import Circuit
            from qibo.derivative import parameter_shift
            # defining an observable
            def hamiltonian(nqubits = 1):
                m0 = (1/nqubits)*hamiltonians.Z(nqubits).matrix
                ham = hamiltonians.Hamiltonian(nqubits, m0)
                return ham
            # defining a dummy circuit
            def circuit(nqubits = 1):
                c = Circuit(nqubits = 1)
                c.add(gates.RY(q = 0, theta = 0))
                c.add(gates.RX(q = 0, theta = 0))
                c.add(gates.M(0))
                return c
            # initializing the circuit
            c = circuit(nqubits = 1)
            # some parameters
            test_params = np.random.randn(2)
            c.set_parameters(test_params)
            test_hamiltonian = hamiltonian()
            # running the psr with respect to the two parameters
            grad_0 = parameter_shift(circuit = c, hamiltonian = test_hamiltonian, parameter_index = 0)
            grad_1 = parameter_shift(circuit = c, hamiltonian = test_hamiltonian, parameter_index = 1)
    """
    """We modify the Qibo implementation to allow multiple parameter_index. For the case where a parameter is reused in different gates"""
    gradient = 0
    try:
        iter(parameter_index) # check if it is iterable
    except:
        parameter_index = [parameter_index]
    if repeated:
        nparam = len(parameter_index)
        parameter_index = parameter_index[:1]
    for parameter_index in parameter_index:
        # inheriting hamiltonian's backend
        backend = hamiltonian.backend

        # getting the gate's type
        gate = circuit.associate_gates_with_parameters()[parameter_index]

        # getting the generator_eigenvalue
        ###try: generator_eigenval = gate.generator_eigenvalue()
        ###except: 
        #generator_eigenval = 0.5   #### CAN'T DO SOMETHING LIKE THIS... ERROR GETS LOGGED ANYWAYS BY QIBO FUNCTION RAISE_ERROR, EVEN IF CAUGHT BY TRY
        # getting the generator_eigenvalue
        generator_eigenval = gate.generator_eigenvalue()

        # defining the shift according to the psr
        s = np.pi / (4 * generator_eigenval)

        # saving original parameters and making a copy
        original = np.asarray(circuit.get_parameters()).copy()
        shifted = original.copy()

        # forward shift and evaluation
        shifted[parameter_index] += s
        circuit.set_parameters(shifted)

        forward = 0
        backward = 0


        if nshots == None:
            forward = hamiltonian.expectation(
                backend.execute_circuit(circuit=circuit, initial_state=initial_state).state()
            )

            # backward shift and evaluation
            shifted[parameter_index] -= 2 * s
            circuit.set_parameters(shifted)

            backward = hamiltonian.expectation(
                backend.execute_circuit(circuit=circuit, initial_state=initial_state).state()
            )

        else:
            
            copied = shifted.copy()

            for _ in range(nruns):

                forward += evaluate(circuit, hamiltonian, nshots) #np.float64(hamiltonian.expectation_from_circuit(circuit, nshots=nshots).real)
                """backend.execute_circuit(
                    circuit=circuit, 
                    initial_state=initial_state, 
                    nshots=nshots
                    ).expectation_from_samples(hamiltonian)"""

                # backward shift and evaluation
                shifted[parameter_index] -= 2 * s
                circuit.set_parameters(shifted)

                backward += evaluate(circuit, hamiltonian, nshots)
                """backend.execute_circuit(
                    circuit=circuit, 
                    initial_state=initial_state, 
                    nshots=nshots
                    ).expectation_from_samples(hamiltonian)  """

                # restoring the original circuit
                shifted = copied.copy()
                circuit.set_parameters(copied)

            forward /= nruns
            backward /= nruns
                
        circuit.set_parameters(original)
        
        gradient += generator_eigenval * (forward - backward) * scale_factor

    if repeated: gradient *= nparam
    return gradient

## Non PSR

In [102]:
import qibo
from qibo import gates, Circuit
from qibo.symbols import I, X, Y, Z
import numpy as np

# Hamiltonian for the TFIsing model
def hamiltonianTFI(N, g):
    ham = I(0)*0
    for q in range(N):
        ham += Z(q)*Z((q+1)%N) + g * X(q)
    return qibo.hamiltonians.SymbolicHamiltonian(-ham)

qibo.gates.RZZ.generator_eigenvalue = lambda self: 0.5
def circ_inv(N, p):
    params = np.random.random((2*p,))*4*np.pi
    circuit_inv = Circuit(N, density_matrix=True)
    for q in range(N):
        circuit_inv.add(gates.H(q))  # WE START WITH |+> state
    for itp in range(p):
        for itzz in range(N):
            circuit_inv.add(gates.RZZ(itzz, (itzz+1)%N, params[2*itp]))
        for itx in range(N):
            circuit_inv.add(gates.RX(itx, params[2*itp + 1]))
    return circuit_inv, params

def circ_noinv(N, p):
    params2 = np.random.random((3*p,))*4*np.pi
    circuit_noinv = Circuit(N, density_matrix=True)
    for q in range(N):
        circuit_noinv.add(gates.H(q))
    for itp in range(p):
        for itzz in range(N):
            circuit_noinv.add(gates.RZZ(itzz, (itzz+1)%N, params2[3*itp]))
        for itx in range(N):
            circuit_noinv.add(gates.RX(itx, params2[3*itp + 1]))
        for ity in range(N):
            circuit_noinv.add(gates.RY(ity, params2[3*itp + 2]))
    return circuit_noinv, params2
    
def circs_shots_noisy_jac(N, p, g = 1, custom_operator=None, nshots=400, noise_map=None, epsilon = 0.02, noise=True):
    if noise_map is None:
        noise_map = {i: list(zip(["X", "Z"], [epsilon, epsilon])) for i in range(N)}

    if custom_operator is None: 
        hamiltonian = hamiltonianTFI(N, g)
    else:
        hamiltonian = custom_operator
    
    circuit_inv, params = circ_inv(N,p)
    circuit_noinv, params2 = circ_noinv(N,p)

    if noise:
        circuit_inv = circuit_inv.with_pauli_noise(noise_map)
        circuit_noinv = circuit_noinv.with_pauli_noise(noise_map)

    def loss(params):
        circuit_inv.set_parameters(np.repeat(params,N))
        return evaluate(circuit_inv, hamiltonian, nshots)#np.float64(hamiltonian.expectation_from_circuit(circuit_inv, nshots=nshots).real)

    def loss2(params):
        circuit_noinv.set_parameters(np.repeat(params,N))
        return evaluate(circuit_noinv, hamiltonian, nshots)#np.float64(hamiltonian.expectation_from_circuit(circuit_noinv, nshots=nshots).real)

    gate_dependence_inv = {i: list(range(i*N, (i+1)*N)) for i in range(0,2*p)}
    gate_dependence_noinv = {i: list(range(i*N, (i+1)*N)) for i in range(0,3*p)}
    def jac_psr_inv(x, *args):
        circuit_inv.set_parameters(np.repeat(x,N))
        return np.array([parameter_shift(circuit_inv, hamiltonian, gate_dependence_inv[param], nshots=nshots, nruns=1, repeated=True) for param in range(len(params))])
    
    def jac_psr_noinv(x, *args):
        circuit_noinv.set_parameters(np.repeat(x,N))
        return np.array([parameter_shift(circuit_noinv, hamiltonian, gate_dependence_noinv[param], nshots=nshots, nruns=1, repeated=True) for param in range(len(params2))])
    
    return circuit_inv, circuit_noinv, params, params2, loss, loss2, jac_psr_inv, jac_psr_noinv


## With noise, some optimizers don't behave as well. For instance BFGS

# We tried implementing a noise resiliant alternative but it does not seem to work too well, because it stops convergence up to random error
# But we know we can keep running Gradient Descent, and stochastically it will converge


# We implement here a simple gradient descent, with a simple learning rate hyperparameter
def optimizer_gds(parameters, loss, gradient, Nepochs=250, lr=1e-2, epochs_print = 50, hyper={}, return_gradients=False):
    loss_list = np.empty((Nepochs,))
    if return_gradients: grad_list = np.empty((Nepochs,len(parameters)))
    for epoch in range(Nepochs):
        grad_val = gradient(parameters)
        parameters = parameters - lr * grad_val
        
        loss_val = loss(parameters)
        if not(epochs_print is None):
            if epoch % epochs_print == 0 or epoch == Nepochs - 1:
                print(f"Epoch {epoch}, Loss: {loss_val}")

        loss_list[epoch] = loss_val
        if return_gradients: grad_list[epoch] = grad_val
    
    if return_gradients: return parameters, loss_list, grad_list
    return parameters, loss_list

In [4]:
c1 = Circuit(4)
c1.add(gates.H(q) for q in range(4))
c1.add(gates.Z(q) for q in range(4))
c1.add(gates.M(q) for q in range(4))

print(c1.with_pauli_noise({q: [('X', 0.00000)] for q in range(4)}))#{q: [('X', 0)]} for q in range(4)))

0: ─H─Z─M─
1: ─H─Z─M─
2: ─H─Z─M─
3: ─H─Z─M─


In [114]:
N = 5
nshots = 50
Nepochs = 50
p = 2
lr = 0.4e-2
rep = 0

epsilon = 0.07

np.random.seed(2000)
init_params = np.random.random((2*p,))*2*np.pi
#print(init_params)


data = []
from time import time
for epsilon in [0, epsilon]:#[0.00000, 0.00001, 0.0001, 0.001, 0.01][:1]:#, 0.001, 0.01, 0.1]:
    for nshots in [None]:#, 400, 4000, 40000, 4000000, None]:
        for noise in [True]:
            #if epsilon != 0: lr *= 100
            print(f"epsilon: {epsilon:.5f}, nshots: {nshots if not(nshots is None) else 0: 6d}, rep: {rep: 2d}")
            #np.random.seed(2025)
            circuit_inv, circuit_noinv, params, params2, loss, loss2, jac1, jac2 = circs_shots_noisy_jac(N, p, nshots=nshots, noise=noise, epsilon=epsilon)
            #print(params)
            params = np.array([3.1754817204963115, 3.5688843461371893, 2.7291836702818535, 2.3745764806624843])#init_params.copy()
            #circuit_inv.draw()
            t0=time()

            _, loss_list, grads = optimizer_gds(params, loss, jac1, Nepochs = Nepochs, epochs_print=None, lr=lr, return_gradients=True)
            print(f"Optimization took {time()-t0:.5f} s")
            data.append({"nshots": nshots, "epsilon":epsilon, "loss": loss_list, "gradients": grads, "noise": noise})

            print(circuit_inv.get_parameters()[::N])
            


epsilon: 0.00000, nshots:      0, rep:  0
Optimization took 2.90714 s
[(3.196585871057331,), (3.571477873443958,), (2.713996310932709,), (2.395758778259591,)]
epsilon: 0.07000, nshots:      0, rep:  0
Optimization took 30.52516 s
[(3.195907455519615,), (3.5692907497128745,), (2.8335277377964436,), (2.3756674767935175,)]


In [38]:
print(np.array(circuit_inv.get_parameters()[::N])[:,0])

[6.27858883 5.39775962 6.31182976 7.32253625 3.28351769 0.74888425]


In [37]:
print(data)

[{'nshots': None, 'epsilon': 0, 'loss': array([-5.12861151, -5.24210343, -5.3354237 , -5.41191173, -5.47454258,
       -5.52588034, -5.56808238, -5.60293235, -5.63188621, -5.65612184,
       -5.67658626, -5.69403797, -5.70908331, -5.72220683, -5.73379609,
       -5.74416184, -5.75355405, -5.76217473, -5.77018803, -5.77772821,
       -5.78490593, -5.79181317, -5.79852707, -5.80511298, -5.81162684,
       -5.81811699, -5.82462566, -5.83119005, -5.83784323, -5.8446148 ,
       -5.85153135, -5.85861687, -5.86589303, -5.87337929, -5.88109308,
       -5.88904982, -5.89726293, -5.90574384, -5.91450187, -5.92354422,
       -5.93287584, -5.94249937, -5.95241501, -5.96262045, -5.97311083,
       -5.98387858, -5.9949135 , -6.00620267, -6.0177305 , -6.0294788 ]), 'gradients': array([[-3.10790936,  0.91453819,  2.21061238,  1.28392664, -2.54021933,
        -3.52617358],
       [-2.79107095,  0.90721183,  1.96108306,  1.2245213 , -2.21008046,
        -3.30257779],
       [-2.49370494,  0.89263474,  

In [71]:
grads

array([[-6.  ,  3.12,  0.24,  1.44],
       [-5.52, -1.92,  2.88, -1.2 ],
       [-2.4 ,  2.16,  0.24, -3.84],
       [-2.16, -0.72,  3.36,  4.56]])

## Plotting

In [18]:
%matplotlib macosx
import matplotlib.pyplot as plt
#plt.ion()
for el in databsc_rem_4qubits_p1:
    nshots = el["nshots"]
    #rep = el["rep"]
    epsilon = el.get("epsilon",0)
    Nepochs = len(el["loss"])
    epoch_range = np.arange(Nepochs)
    error = np.sqrt(2*N/nshots) if nshots is not None else 0
    loss_list = el["loss"]
    grads = el.get("gradients",None)
    noise = el.get("noise", None)
    #plt.title(f"N: {N}, p: {p}\nnshots: {nshots}, epsilon: {epsilon: .5f}")#, epsilon: {epsilon}")#, rep: {rep}, epsilon: {epsilon}")
    plt.title("Quantum chip loss evolution")
    plt.ylabel("Loss Function")
    plt.xlabel("Epoch")
    plt.plot(epoch_range, loss_list)
    plt.fill_between(epoch_range, loss_list+error, loss_list-error, alpha=0.1)
    #plt.plot([0,Nepochs-1],[min(loss_list)]*2, '--c')
    plt.plot([0,Nepochs-1],[-6.472135954999578]*2, ':')
    ax_grads = plt.twinx()
    ax_grads.set_ylabel("Gradients")
    max_grads = np.average(grads, axis=1)
    ax_grads.plot(epoch_range, max_grads, '-.y')
    ax_grads.fill_between(epoch_range, max_grads+error/np.sqrt(2), max_grads-error/np.sqrt(2), alpha=0.1)

    plt.show(block=True)


# MISSING:
#   ALSO PLOT WITH axis.twinx or something like that GRADIENTS AND HOW NOISE COMPARES TO THEM
    

    

In [43]:
# data absc
databsc = [{'nshots': 400, 'loss': np.array([-0.42 , -0.17 , -0.105,  0.04 , -0.035, -0.17 , -0.11 , -0.34 ,
       -0.435, -0.16 , -0.14 , -0.225,  0.135, -0.105, -0.365]), 'gradients': np.array([[ 1.02500000e+00,  2.25000000e-01, -5.62500000e-01,
         2.75000000e-01],
       [ 1.67500000e+00, -6.37500000e-01,  1.50000000e-01,
         6.12500000e-01],
       [ 6.12500000e-01,  3.62500000e-01, -1.03750000e+00,
         5.12500000e-01],
       [-8.50000000e-01,  8.25000000e-01, -1.87500000e-01,
         1.87500000e-01],
       [-4.75000000e-01, -1.50000000e-01, -5.62500000e-01,
         1.75000000e-01],
       [-3.87500000e-01,  5.75000000e-01, -7.75000000e-01,
        -6.25000000e-02],
       [ 7.50000000e-01,  9.50000000e-01, -8.37500000e-01,
         1.87500000e-01],
       [ 7.37500000e-01,  1.62500000e-01, -1.10000000e+00,
         1.07500000e+00],
       [ 1.00000000e+00,  0.00000000e+00, -9.50000000e-01,
        -1.37500000e-01],
       [ 6.37500000e-01,  2.50000000e-02, -1.33750000e+00,
         1.87500000e-01],
       [ 2.75000000e-01, -6.24500451e-16, -1.17500000e+00,
         1.50000000e-01],
       [ 8.50000000e-01, -5.62500000e-01, -8.12500000e-01,
         1.02500000e+00],
       [ 1.25000000e-02, -5.50000000e-01, -4.12500000e-01,
         5.00000000e-02],
       [ 1.47500000e+00,  1.12500000e-01,  1.00000000e-01,
         8.75000000e-01],
       [ 7.62500000e-01,  7.25000000e-01,  1.12500000e-01,
         3.12500000e-01]]), 'noise': False}]

databsc[0]["epsilon"] = 0.

In [139]:
databsc_rem = [{'nshots': 1000, 'loss': np.array([-0.58281466, -0.62147871, -0.65534908, -0.47591641, -0.75974995,
       -0.85999552, -1.04324051, -0.90908734, -0.69335976, -0.60419162,
       -0.90422048, -0.45902153, -0.94049668, -0.72727865, -0.59635191]), 'gradients': np.array([[-1.44220187e+00,  3.24549144e-02, -2.70682556e+00,
        -5.43004305e-01],
       [-1.07459603e+00,  1.24491789e+00, -1.11424303e+00,
        -3.07355097e-01],
       [-1.34700959e+00, -2.02333721e-02, -9.63941711e-01,
         2.19829291e-03],
       [-1.89663901e-01, -8.15402883e-01, -1.62444046e+00,
        -1.71643354e+00],
       [-6.60701549e-01,  5.47296744e-02, -1.63080020e+00,
        -1.94061524e+00],
       [-1.44621530e+00, -1.22999991e-02, -1.91458007e+00,
         1.92279535e-01],
       [-2.22769807e-01,  2.16910356e-01, -1.94778811e+00,
        -1.30820009e+00],
       [ 3.58838603e-01, -1.05996733e+00, -2.03014878e+00,
        -5.20340472e-01],
       [-1.54240677e+00,  5.86636229e-01, -1.02801508e+00,
        -8.22794137e-01],
       [-7.22415338e-01, -8.05472660e-03, -2.29132709e+00,
        -5.52106570e-01],
       [-8.14511472e-01, -2.83550590e-01, -9.56243912e-01,
        -1.09862844e+00],
       [ 6.60630298e-01, -3.17714307e-01, -1.09436656e+00,
        -1.79662978e+00],
       [ 2.01470506e-01,  7.44045498e-01, -1.11243719e+00,
         7.67464257e-02],
       [-4.69507977e-01,  9.02878250e-01, -2.00933287e+00,
        -1.18121155e-01],
       [-5.71012728e-01,  7.54885762e-01, -1.72677915e+00,
        -9.34216148e-01]]), 'noise': False}]

databsc_rem[0]["epsilon"] = 0.

rem_coefficients = [[0.491, 0.38249999999999995], [0.50025, 0.35675], [0.47350000000000003, 0.3025], [0.48924999999999996, 0.31725000000000003], [0.49024999999999996, 0.36675]]


nshots_rem = 2000
for q in range(5):
    p01 = 1- rem_coefficients[q][0]-rem_coefficients[q][1]
    p10 = rem_coefficients[q][0]-rem_coefficients[q][1]
    print(f"Qubit {q}: P(0|1) = {p01} +- {p01*(1-p01)/np.sqrt(nshots_rem)}\n         P(1|0) = {p10} +- {p10*(1-p10)/np.sqrt(nshots_rem)}")

Qubit 0: P(0|1) = 0.12650000000000006 +- 0.002470804803607775
         P(1|0) = 0.10850000000000004 +- 0.0021628982431060534
Qubit 1: P(0|1) = 0.14300000000000002 +- 0.002740323667105767
         P(1|0) = 0.14349999999999996 +- 0.002748300839615497
Qubit 2: P(0|1) = 0.22399999999999998 +- 0.003886822801209234
         P(1|0) = 0.17100000000000004 +- 0.0031698276042239275
Qubit 3: P(0|1) = 0.1935 +- 0.0034895573741566778
         P(1|0) = 0.17199999999999993 +- 0.0031845185708360996
Qubit 4: P(0|1) = 0.14300000000000002 +- 0.002740323667105767
         P(1|0) = 0.12349999999999994 +- 0.0024204932741140276


In [135]:
databsc_rem_4 = [{'nshots': 2000, 'loss': np.array([-0.11608069, -0.40747727, -0.27771414, -0.32701962, -0.59100034,
       -0.23878684, -0.36339266, -0.45268504, -0.28196428, -0.34698439,
       -0.31896352, -0.3621411 , -0.39144895, -0.27907616, -0.44459749]), 'gradients': np.array([[ 0.70683335,  0.4664251 ,  0.81158743, -0.16391852],
       [ 1.05026853,  0.16947027, -0.07540248,  0.39785574],
       [ 0.92334054,  0.46347355,  0.63114191, -0.35528062],
       [ 0.70696113,  0.60767384,  0.0292773 , -0.43814754],
       [ 0.23675895,  0.46059989,  0.18429616, -0.18599539],
       [ 1.11396948, -0.06901642,  0.37682756, -0.03808154],
       [ 0.68266757, -0.02487311,  0.42631735,  0.51071667],
       [ 1.03005504, -0.12027182,  0.26974442,  0.42758487],
       [-0.10019078,  0.04465778,  0.52145312, -0.28523743],
       [ 0.98256855,  0.3782085 ,  0.75028198,  0.1321615 ],
       [ 0.61572583,  0.2156212 ,  0.80278408,  0.17274965],
       [ 0.50866325, -0.37191233,  0.63943337, -0.26882946],
       [ 0.70103168,  0.60628623, -0.06520464, -0.46825271],
       [-0.08771344,  0.36383395,  0.02461151, -0.16237283],
       [ 0.15964897,  0.3991636 ,  0.046889  ,  0.44944554]]), 'noise': False}]
[[0.501, 0.373], [0.498, 0.3335], [0.4785, 0.32649999999999996], [0.50475, 0.33325], [0.49175, 0.37275]]

databsc_rem_4[0]["epsilon"] = 0.

In [16]:
import numpy as np

N = 4
databsc_rem_4qubits_p1 = [{'nshots': 3000, 'loss': np.array([-0.71828238, -0.87660908, -1.04674528, -1.07529737, -0.76450407,
       -1.08388104, -1.3235531 , -1.14833925, -0.98853462, -1.27808984,
       -1.1517414 , -1.21833345, -0.99902537, -1.1643264 , -1.10757196]), 'gradients': np.array([[-3.70178474e+00,  2.94938238e+00],
       [-3.42460134e+00,  1.43040992e+00],
       [-3.37545358e+00,  2.02440674e+00],
       [-2.91637619e+00,  1.14127361e+00],
       [-2.42493812e+00,  1.68682452e+00],
       [-2.59566628e+00,  1.46416327e+00],
       [-2.29505411e+00,  4.58997235e-01],
       [-1.91736804e+00,  8.99611595e-01],
       [-1.97097536e+00,  2.91166772e-01],
       [-2.04281344e+00,  8.19129968e-01],
       [-1.46143979e+00,  6.21844918e-01],
       [-8.72728367e-01,  6.28352969e-01],
       [-7.43934233e-01,  6.59413794e-01],
       [-1.50472375e+00,  2.24138691e-03],
       [-5.71528992e-01,  4.38293566e-01]]), 'noise': False}]
[[0.496, 0.36450000000000005], [0.507, 0.29300000000000004], [0.52075, 0.29075], [0.47525, 0.32125], [0.48650000000000004, 0.366]]

[[0.496, 0.36450000000000005],
 [0.507, 0.29300000000000004],
 [0.52075, 0.29075],
 [0.47525, 0.32125],
 [0.48650000000000004, 0.366]]

In [14]:
N=4
databsc_rem_4qubits_p1_simulated = [{'nshots': 3000, 'loss': np.array([-3.88420189, -3.88475124, -4.01652678, -3.85345196, -3.92933258,
       -3.8910827 , -3.79727578, -3.80344567, -3.77903465, -3.73031402,
       -3.83905502, -3.8783796 , -3.60935803, -3.81032643, -3.69790925,
       -3.74336726, -3.63147951, -3.60265075, -3.74823403, -3.60073697,
       -3.72605748, -3.68033272, -3.39656794, -3.63890823, -3.35104412,
       -3.56968409, -3.61919396, -3.62124963, -3.49523535, -3.63890509]), 'gradients': np.array([[ 0.55695805, -1.85112222],
       [ 0.28168189, -1.71294423],
       [-0.01740421, -1.8407677 ],
       [-0.29142239, -1.00730265],
       [ 0.31948827, -0.85182734],
       [ 0.18773211, -1.85955886],
       [ 0.31015242, -1.5662584 ],
       [ 0.22624762, -0.8569213 ],
       [-0.09546644, -1.11090913],
       [ 0.01609882, -1.03682751],
       [ 0.2968007 , -1.16923566],
       [-0.02451757, -0.99349959],
       [ 0.0057048 , -1.03846685],
       [-0.12256055, -0.42685592],
       [-0.16845755, -0.78206476],
       [-0.11872509, -0.41269599],
       [-0.14416819, -0.95069836],
       [ 0.1583888 , -0.42858053],
       [-0.04417847, -1.08836174],
       [-0.16451542, -0.60138995],
       [ 0.22718795, -0.92483026],
       [-0.17403678, -0.65802903],
       [-0.15009905, -1.11093353],
       [-0.51418545, -0.40150014],
       [-0.30375509, -0.42838285],
       [-0.01475618, -0.34605084],
       [-0.31480722, -0.43722756],
       [-0.44109956, -0.44444768],
       [ 0.12100764, -0.36569216],
       [-0.45609224, -0.55030736]]), 'noise': False}]

In [9]:
dataBSC_noaction_4qubits_p1 = [{'loss': np.array([-6.772000000000001, -6.797333333333334, -6.733333333333334, -6.742000000000001, -6.856666666666667, -6.792, -6.828666666666667, -6.775333333333333, -6.797333333333334, -6.758666666666667, -6.8486666666666665, -6.746, -6.831333333333333, -6.721333333333334, -6.774, -6.826666666666666, -6.741999999999999, -6.75, -6.772666666666667, -6.821333333333335, -6.799333333333333, -6.747333333333334, -6.731333333333334, -6.730666666666668, -6.742000000000001, -6.816000000000001, -6.7120000000000015, -6.792666666666667, -6.804, -6.753333333333334, -6.777333333333334, -6.764000000000001, -6.769333333333334, -6.78, -6.747333333333334, -6.720000000000001, -6.768666666666666, -6.786666666666667, -6.770666666666667, -6.756666666666668, -6.813333333333334, -6.778666666666667, -6.827333333333334, -6.790666666666668, -6.7826666666666675, -6.7620000000000005, -6.766, -6.754666666666667, -6.740666666666666, -6.772666666666666]), 'state': [], 'nshots': 3000}]
N = 4

In [12]:
# BAD DATA WITH NOISE 25epochs
#data2 = data.copy()

In [69]:
# DATA WITHOUT ERROR NOISE 50epochs
#dataclean = data.copy()

In [109]:
# Data with noise, but not too god data 40epochs
#datanoise = data.copy()

In [44]:
# DAta wihout error noise 40epochs
#dataclean2 = data.copy()

NameError: name 'dataclean2' is not defined